In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from PyPDF2 import PdfReader
import gradio as gr

e:\ML Projects\Autogen-AgentAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override = True)
client = OpenAI()

In [3]:
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"

In [5]:
def push(message):
    print(f"Push: {message}")
    payload = {"user" : pushover_user, "token" : pushover_token, "message" : message}
    requests.post(pushover_url, data=payload)

In [6]:
push("Hey!")

Push: Hey!


In [8]:
def record_user_details(email, name = "Name not provided", notes = "Not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [9]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I could not answer")
    return {"recorded": "ok"}

This is a tool

In [10]:
record_user_details_json = {
    "name" : "record_user_details",
    "description" : "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "email" : {
                "type" : "string",
                "description" : "The email address this user"
            },
            "name" : {
                "type" : "string",
                "description" : "The user's name, if they provided it"
            },
            "notes" : {
                "type" : "string",
                "description" : "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required" : ['email'],
        "additionalProperties" : False
    }
}

In [11]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [12]:
tools = [{"type" : "function", "function" : record_user_details_json},
         {"type" : "function", "function" : record_unknown_question_json}]

In [13]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['question

In [14]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.load(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)
    return result

In [15]:
globals()["record_unknown_question"]("This is a really hard question")

Push: Recording This is a really hard question asked that I could not answer


{'recorded': 'ok'}

In [19]:
# Writing the above function in a more elegant way 

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [23]:
reader = PdfReader("../me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("../me/summary.txt", mode='r', encoding='utf-8') as f:
    summary = f.read()

name = "Udit Sharma"

In [24]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [31]:
def chat(message, history):
    messages = [{"role" : "system", "content" : system_prompt}] + history + [{"role" : "user", "content" : message}]
    done = False
    while not done:
        response = client.chat.completions.create(model = 'gpt-4o-mini', messages = messages, tools=tools)
        finish_reason = response.choices[0].finish_reason
        print(f"finish_reason: {finish_reason}")

        # response might be the final response to be sent to user or it might indicate the need to use a particular tool 
        if finish_reason == 'tool_calls':
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [32]:
gr.ChatInterface(chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


finish_reason: tool_calls
Tool called: record_unknown_question
Push: Recording What is Udit Sharma's leetcode id? asked that I could not answer
finish_reason: stop
finish_reason: tool_calls
Tool called: record_user_details
Push: Recording interest from Name not provided with email heya@heya.com and notes Not provided
finish_reason: stop
finish_reason: tool_calls
Tool called: record_user_details
Push: Recording interest from Rayz with email rays@gmail.com and notes Interested in internship opportunities in my company.
Tool called: record_unknown_question
Push: Recording How to apply for an internship at my company? asked that I could not answer
finish_reason: stop
